# Language Model: flat data, without context

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mostly-ai/mostlyai-engine/blob/main/examples/language.ipynb)

In [4]:
from pathlib import Path
import pandas as pd
from mostlyai import engine

# init workspace and logging
# ws = Path("ws-language-flat")
ws = Path("ws-language-categorical-flat")
engine.init_logging()

# # load original data
# url = "https://github.com/mostly-ai/public-demo-data/raw/refs/heads/dev/arxiv"
# # trn_df = pd.read_parquet(f"{url}/synthetic-data-papers.parquet")[['category', 'title']]
# trn_df = pd.read_parquet(f"{url}/synthetic-data-papers.parquet")[['category']]

# execute the engine steps
# engine.split(                         # split data as PQT files for `trn` + `val` to `{ws}/OriginalData/tgt-data`
#     workspace_dir=ws,
#     tgt_data=trn_df,
#     # model_type="LANGUAGE",
#     tgt_encoding_types={"category": "LANGUAGE_CATEGORICAL"},
# )
# engine.analyze(workspace_dir=ws)      # generate column-level statistics to `{ws}/ModelStore/tgt-stats/stats.json`
# engine.encode(workspace_dir=ws)       # encode training data to `{ws}/OriginalData/encoded-data`
# engine.train(                         # train model and store to `{ws}/ModelStore/model-data`
#     workspace_dir=ws,
#     model="MOSTLY_AI/LSTMFromScratch-3m",  # use a light-weight LSTM model, trained from scratch (GPU recommended)
#     # model="microsoft/phi-1.5",           # or alternatively use a HF-hosted LLM model (GPU required)
#     max_training_time=0.2,                  # limit TRAIN to 10 minute for demo purposes
# )
engine.generate(                      # use model to generate synthetic samples to `{ws}/SyntheticData`
    workspace_dir=ws, 
    sample_size=100,
)

[2025-02-05 13:43:49,105] INFO   : TRAIN_LANGUAGE started
[2025-02-05 13:43:49,115] INFO   : numpy=1.26.4, pandas=2.2.3
[2025-02-05 13:43:49,118] INFO   : torch=2.5.1, opacus=1.5.2
[2025-02-05 13:43:49,123] INFO   : transformers=4.46.3, peft=0.11.1
[2025-02-05 13:43:49,124] INFO   : device=device(type='cpu')
[2025-02-05 13:43:49,126] INFO   : bf16_supported=False
[2025-02-05 13:43:49,126] INFO   : use_mixed_precision=False
[2025-02-05 13:43:49,127] INFO   : model_id='MOSTLY_AI/LSTMFromScratch-3m'
[2025-02-05 13:43:49,127] INFO   : enable_flexible_generation=True
[2025-02-05 13:43:49,128] INFO   : max_training_time=12.0s
[2025-02-05 13:43:49,128] INFO   : max_epochs=100.0
[2025-02-05 13:43:49,129] INFO   : with_dp=False
[2025-02-05 13:43:49,130] INFO   : model_state_strategy=<ModelStateStrategy.reset: 'RESET'>


0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


[2025-02-05 13:44:01,027] INFO   : create training model
[2025-02-05 13:44:01,031] INFO   : model weights not found; change strategy from ModelStateStrategy.reset to RESET
[2025-02-05 13:44:01,032] INFO   : model_state_strategy=<ModelStateStrategy.reset: 'RESET'>
[2025-02-05 13:44:01,033] INFO   : clear existing checkpoint files
[2025-02-05 13:44:01,035] INFO   : start training progress from epoch=0.0, steps=0
[2025-02-05 13:44:01,233] INFO   : model loading time: 0.20s
[2025-02-05 13:44:01,234] INFO   : no_of_model_params=595591
[2025-02-05 13:44:01,235] INFO   : no_of_trainable_model_params=595591
[2025-02-05 13:44:01,235] INFO   : tokenizer=LlamaTokenizerFast(name_or_path='', vocab_size=135, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>"

In [ ]:
syn_tgt_df = pd.read_parquet(ws / "SyntheticData") # load synthetic data
syn_tgt_df.head(5)